In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.dates as mdates

Strategies:
    1 - Simple MA Crossover strategy
    2 - Go long  when: 
            The S&P 500 Index is above its 200-day moving average;
            The 2-period RSI of the S&P 500 Index closes below 5;
        Exit the trade when:
            Exit when the S&P closes above its 5-period moving average.
    3 - Mean Reversion Strategy:
    

In [2]:
aapl_min_final = pd.read_csv('/Users/raphaelravinet/Code/algo_trading/aapl_minute_final.csv', index_col=0)

In [3]:
aapl_daily = pd.read_csv('/Users/raphaelravinet/Code/algo_trading/aapl_daily_final.csv', index_col=0)
aapl_daily


,id,date.1,timestamp,ticker,open,high,low,close,volume,vwap,...,sma_slope_30,ema_slope_30,sma_slope_50,ema_slope_50,sma_slope_100,ema_slope_100,sma_slope_200,ema_slope_200,sma_slope_300,ema_slope_300
date,,,,,,,,,,,,,,,,,,,,,
2020-01-06,4605,2020-01-06,1578286800000,AAPL,73.4475,74.9900,73.1875,74.9500,118518576.0,74.4499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,4606,2020-01-07,1578373200000,AAPL,74.9600,75.2250,74.3700,74.5975,111510620.0,74.7311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,4607,2020-01-08,1578459600000,AAPL,74.2900,76.1100,74.2890,75.7975,132363784.0,75.2914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-09,4608,2020-01-09,1578546000000,AAPL,76.8088,77.6075,76.5500,77.4075,170486168.0,77.1610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-10,4609,2020-01-10,1578632400000,AAPL,77.6500,78.1675,77.0625,77.5825,140869088.0,77.6729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26,5751,2024-07-26,1721966400000,AAPL,218.7000,219.4900,216.0100,217.9600,39827645.0,217.8992,...,0.264850,0.036604,0.330544,0.167794,0.210758,0.198719,0.114984,0.151804,0.084057,0.124633
2024-07-29,5752,2024-07-29,1722225600000,AAPL,216.9600,219.3000,215.7500,218.2400,35153729.0,218.1319,...,0.159067,-0.007568,0.301534,0.134429,0.217707,0.180246,0.107467,0.142635,0.081918,0.118545
2024-07-30,5753,2024-07-30,1722312000000,AAPL,219.1900,220.3250,216.1200,218.8000,40681625.0,218.4059,...,0.090907,-0.003326,0.285397,0.131324,0.235160,0.177600,0.104841,0.141670,0.081078,0.118070


In [4]:
aapl_min_final

,id,date.1,timestamp,ticker,open,high,low,close,volume,vwap,...,sma_slope_30,ema_slope_30,sma_slope_50,ema_slope_50,sma_slope_100,ema_slope_100,sma_slope_200,ema_slope_200,sma_slope_300,ema_slope_300
date,,,,,,,,,,,,,,,,,,,,,
2020-01-06 04:00:00,1,2020-01-06 04:00:00,1578301200000,AAPL,73.8575,73.8775,73.5500,73.5500,7752.0,73.7602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 04:01:00,2,2020-01-06 04:01:00,1578301260000,AAPL,73.5500,73.5500,73.5500,73.5500,1388.0,73.5683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 04:02:00,3,2020-01-06 04:02:00,1578301320000,AAPL,73.6950,73.6950,73.6925,73.6925,1492.0,73.6957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 04:05:00,4,2020-01-06 04:05:00,1578301500000,AAPL,73.6925,73.7375,73.6925,73.7375,2400.0,73.7165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 04:06:00,5,2020-01-06 04:06:00,1578301560000,AAPL,73.6800,73.7500,73.6800,73.7500,2736.0,73.7330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01 11:04:00,914404,2024-08-01 11:04:00,1722524640000,AAPL,220.5200,220.6199,220.1500,220.1501,118141.0,220.3381,...,-0.013568,-0.023907,-0.026616,-0.021039,-0.017485,-0.016950,-0.009530,-0.011382,-0.004153,-0.008188
2024-08-01 11:05:00,914405,2024-08-01 11:05:00,1722524700000,AAPL,220.1800,220.4000,220.1500,220.1850,108400.0,220.2574,...,-0.016056,-0.025519,-0.027322,-0.022129,-0.017811,-0.017579,-0.009875,-0.011752,-0.004215,-0.008456
2024-08-01 11:06:00,914406,2024-08-01 11:06:00,1722524760000,AAPL,220.1700,220.6400,220.1200,220.5400,93030.0,220.4027,...,-0.014628,-0.023621,-0.026419,-0.021110,-0.017525,-0.017155,-0.009903,-0.011597,-0.004173,-0.008374


In [5]:
class Strategies:
    def __init__(self,df):
        self.df = df.copy()
        
    def Strategy1(self):
        self.df['strategy_1_position'] = np.where(self.df['sma_50'] > self.df['sma_200'], 1, -1)
        return self.df       
    
    def Strategy2(self):
        self.df['strategy_2_position'] = 0
        self.df['strategy_2_position'] = np.where((self.df['close'] < self.df['lower_band']) & (self.df['IBS'] < 0.3),1,
                                       np.where((self.df['close'] > self.df['high'].shift(1)) | (self.df['close'] < self.df['sma_300']),-1,
                                                self.df['position'].shift(1)
                                                )
                                       )
        return self.df
    
    def Strategy3(self):
        self.df['strategy_3_position'] = np.where((self.df['close'] > self.df['sma_200']) & self.df['RSI_2'] < 5, 1,
                                       np.where(self.df['close'] > self.df['sma_5'], -1,
                                                self.df['position'].shift(1)))
        return self.df
    
    
    def compute_return(self):
        self.df[f'strategy_return'] = self.df['position'].shift(1) * self.df['log_ret']
        return self.df 

In [6]:
strategy = Strategy3(aapl_daily)

In [7]:
final_df = strategy.manage_position()

In [8]:
final_df = strategy.compute_return()

In [9]:
final_df.index = pd.to_datetime(final_df.index)

In [10]:
final_df = final_df.iloc[203:]

In [11]:
final_df

,id,date.1,timestamp,ticker,open,high,low,close,volume,vwap,...,sma_slope_50,ema_slope_50,sma_slope_100,ema_slope_100,sma_slope_200,ema_slope_200,sma_slope_300,ema_slope_300,position,strategy_return
date,,,,,,,,,,,,,,,,,,,,,
2020-10-23,4808,2020-10-23,1603425600000,AAPL,116.39,116.550,114.2800,115.04,82304645.0,114.9857,...,0.068868,0.083805,0.340719,0.188991,0.228632,0.213311,NaN,NaN,1,-0.006153
2020-10-26,4809,2020-10-26,1603684800000,AAPL,114.01,116.550,112.8800,115.05,111849857.0,114.6132,...,0.040241,0.057157,0.330224,0.172247,0.219780,0.203689,NaN,NaN,1,0.000087
2020-10-27,4810,2020-10-27,1603771200000,AAPL,115.49,117.280,114.5399,116.60,92168772.0,116.3162,...,0.017011,0.050590,0.323589,0.166188,0.212379,0.199934,NaN,NaN,1,0.013382
2020-10-28,4811,2020-10-28,1603857600000,AAPL,115.05,115.430,111.1000,111.20,143937823.0,112.7270,...,-0.003849,0.022641,0.309135,0.148591,0.204032,0.189734,NaN,NaN,1,-0.047419
2020-10-29,4812,2020-10-29,1603944000000,AAPL,112.37,116.930,112.2000,115.32,146124673.0,114.5124,...,-0.012945,0.010434,0.293573,0.139309,0.200970,0.184080,NaN,NaN,1,0.036380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26,5751,2024-07-26,1721966400000,AAPL,218.70,219.490,216.0100,217.96,39827645.0,217.8992,...,0.330544,0.167794,0.210758,0.198719,0.114984,0.151804,0.084057,0.124633,1,0.002159
2024-07-29,5752,2024-07-29,1722225600000,AAPL,216.96,219.300,215.7500,218.24,35153729.0,218.1319,...,0.301534,0.134429,0.217707,0.180246,0.107467,0.142635,0.081918,0.118545,1,0.001284
2024-07-30,5753,2024-07-30,1722312000000,AAPL,219.19,220.325,216.1200,218.80,40681625.0,218.4059,...,0.285397,0.131324,0.235160,0.177600,0.104841,0.141670,0.081078,0.118070,1,0.002563


In [ ]:
plt.figure(figsize=(14,8))

plt.xticks(rotation=45)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))

plt.plot(final_df.index, final_df['close'], label='AAPL', color='blue')
plt.plot(final_df['sma_50'], label='50 Day SMA', color='orange')
plt.plot(final_df['sma_200'], label='200 Day SMA', color='green')

plt.title('AAPL 50 and 200 Day SMA')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

# Show the plot
plt.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['close'], mode = 'lines', name = 'AAPL', line = dict(color = 'blue')))
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['sma_50'], mode = 'lines', name = '50 Day SMA', line = dict(color = 'orange')))
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['sma_200'], mode = 'lines', name = '200 Day SMA', line = dict(color = 'green')))
fig.add_trace(go.Scatter)
fig.update_layout(
    title="AAPL 50 and 200 Day SMA",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis=dict(
        tickformat="%Y-%m",
        tickangle=45,
    ),
    width=1000,
    height=600
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=final_df.index, y=final_df['close'], mode='lines', name='AAPL', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=final_df.index, y=final_df['sma_50'], mode='lines', name='50 Day SMA', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=final_df.index, y=final_df['sma_200'], mode='lines', name='200 Day SMA', line=dict(color='green')))

# shaded regions for positions
for i in range(1, len(final_df)):
    if final_df['position'].iloc[i] == 1:
        fig.add_shape(type="rect",
                      x0=final_df.index[i-1], y0=min(final_df['close']), 
                      x1=final_df.index[i], y1=max(final_df['close']),
                      fillcolor="green", opacity=0.3, line_width=0)
    elif final_df['position'].iloc[i] == -1:
        fig.add_shape(type="rect",
                      x0=final_df.index[i-1], y0=min(final_df['close']), 
                      x1=final_df.index[i], y1=max(final_df['close']),
                      fillcolor="red", opacity=0.3, line_width=0)


fig.update_layout(
    title="AAPL 50 and 200 Day SMA with Positions",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis=dict(
        tickformat="%Y-%m",
        tickangle=45,
    ),
    width=1000,
    height=600
)

fig.show()
